<a href="https://colab.research.google.com/github/Pravalika-26/deep-learning/blob/main/svhm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install tensorflow numpy matplotlib seaborn


In [9]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.io import loadmat
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Load SVHN dataset
train_data = loadmat('http://ufldl.stanford.edu/housenumbers/train_32x32.mat')
test_data = loadmat('http://ufldl.stanford.edu/housenumbers/test_32x32.mat')

# Extract images and labels
X_train = np.moveaxis(train_data['X'], -1, 0)  # Shape (73257, 32, 32, 3)
y_train = train_data['y'].flatten()            # Shape (73257,)
X_test = np.moveaxis(test_data['X'], -1, 0)    # Shape (26032, 32, 32, 3)
y_test = test_data['y'].flatten()              # Shape (26032,)

# Convert label 10 to 0
y_train[y_train == 10] = 0
y_test[y_test == 10] = 0

# Normalize the images
X_train, X_test = X_train / 255.0, X_test / 255.0

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Splitting the dataset
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10, random_state=42)

# Plot some sample images
fig, axes = plt.subplots(1, 10, figsize=(15, 5))
for i, ax in enumerate(axes):
    ax.imshow(X_train[i])
    ax.set_title(np.argmax(y_train[i]))
    ax.axis('off')
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'http://ufldl.stanford.edu/housenumbers/train_32x32.mat'

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.regularizers import l2

def create_model(hidden_layers=3, hidden_units=64, activation='relu', optimizer='adam', l2_reg=0.0005):
    model = Sequential()
    model.add(Flatten(input_shape=(32, 32, 3)))

    # Adding hidden layers
    for _ in range(hidden_layers):
        model.add(Dense(hidden_units, activation=activation, kernel_regularizer=l2(l2_reg)))

    # Output layer
    model.add(Dense(10, activation='softmax'))

    # Compile model
    opt_dict = {
        "sgd": SGD(),
        "adam": Adam(),
        "rmsprop": RMSprop()
    }
    model.compile(optimizer=opt_dict[optimizer], loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
def train_and_evaluate(hidden_layers, hidden_units, activation, optimizer, l2_reg, epochs=10, batch_size=32):
    model = create_model(hidden_layers, hidden_units, activation, optimizer, l2_reg)
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

    # Evaluate on test set
    test_loss, test_acc = model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {test_acc:.4f}")

    return history, test_acc

# Example Training
history, test_acc = train_and_evaluate(hidden_layers=3, hidden_units=64, activation='relu',
                                       optimizer='adam', l2_reg=0.0005, epochs=10, batch_size=32)


In [ ]:
configs = [
    (3, 64, 'relu', 'adam', 0.0005),
    (4, 128, 'relu', 'rmsprop', 0.0005),
    (5, 32, 'sigmoid', 'sgd', 0.0)
]

results = {}
for config in configs:
    print(f"\nTraining with {config}")
    history, test_acc = train_and_evaluate(*config, epochs=10, batch_size=32)
    results[config] = test_acc

# Print results
for config, acc in results.items():
    print(f"Config {config} -> Test Accuracy: {acc:.4f}")


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Predict on test data
y_pred = np.argmax(model.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Plot
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=range(10), yticklabels=range(10))
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()
